<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/FeatureCollection/reducing_feature_collection.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/FeatureCollection/reducing_feature_collection.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-notebooks/master?filepath=FeatureCollection/reducing_feature_collection.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/FeatureCollection/reducing_feature_collection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The magic command `%%capture` can be used to hide output from a specific cell. Uncomment these lines if you are running this notebook for the first time.

In [ ]:
# %%capture
# !pip install earthengine-api
# !pip install geehydro

Import libraries

In [ ]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for the first time or if you are getting an authentication error.  

In [ ]:
# ee.Authenticate()
ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [ ]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [ ]:
def areaDiff(feature):
  area = feature.geometry().area().divide(1000 * 1000)
  # Compute the differece between computed area and the area property.
  diff = area.subtract(ee.Number.parse(feature.get('areasqkm')))
  # Return the feature with the squared difference set to the 'diff' property.
  return feature.set('diff', diff.pow(2))


# Load watersheds from a data table and filter to the continental US.
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06') \
  .filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29))

# This function computes the squared difference between an area property
# and area computed directly from the feature's geometry.
# areaDiff = function(feature) {
#   # Compute area in sq. km directly from the geometry.
#   area = feature.geometry().area().divide(1000 * 1000)
#   # Compute the differece between computed area and the area property.
#   diff = area.subtract(ee.Number.parse(feature.get('areasqkm')))
#   # Return the feature with the squared difference set to the 'diff' property.
#   return feature.set('diff', diff.pow(2))
# }

# Calculate RMSE for population of difference pairs.
rmse = ee.Number(
  # Map the difference function over the collection.
  sheds.map(areaDiff)
  # Reduce to get the mean squared difference. \
  .reduceColumns(ee.Reducer.mean(), ['diff']) \
  .get('mean')
) \
.sqrt()

# Print the result.
print('RMSE=', rmse.getInfo())



## Display Earth Engine data layers 

In [ ]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map